In [2]:
# Do imports
import os
import logging
import sys
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Enable tracing
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
# Add phi3:3.8b LLM model
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="phi3:3.8b",
)

In [4]:
# Test LLM model is working
response_message = model.invoke("What is the capital of France?")

print(response_message.content)

The capital of France is Paris. It'dis known for its significant influence on culture, art, fashion and cuisine; it also plays a major role in global economy as one of world’s leading financial centers - the "City of Light" got its name because Paris was an early adopter of street lighting during the Industrial Revolution.


In [9]:
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import random

class QuizChatbot:
    def __init__(self, csv_path: str):
        # Load the CSV data
        self.df = pd.read_csv(csv_path)
        self.categories = self.df['Category'].unique().tolist()
        
        # Initialize the Ollama model
        self.llm = ChatOllama(
            model="phi3:3.8b",
            temperature=0
        )
        
        # Initialize state variables
        self.current_question = None
        self.current_answer = None
        
    def get_categories(self) -> str:
        """Return formatted list of categories"""
        return "\n".join([f"{i+1}. {cat}" for i, cat in enumerate(self.categories)])
    
    def select_question(self, category: str) -> tuple:
        """Select a random question from the specified category"""
        category_questions = self.df[self.df['Category'] == category]
        if len(category_questions) == 0:
            return None, None
        
        question_row = category_questions.sample(n=1).iloc[0]
        return question_row['Question'], question_row['Answer']
    
    def verify_answer(self, user_answer: str, correct_answer: str) -> bool:
        """Use the LLM to verify if the answer is correct"""
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are an answer verification expert. 
            Compare the user's answer with the correct answer and determine if they are semantically equivalent.
            Respond with only 'True' if correct or 'False' if incorrect."""),
            HumanMessage(content=f"""
            Correct answer: {correct_answer}
            User's answer: {user_answer}
            Are these answers semantically equivalent?
            """)
        ])
        
        response = self.llm.invoke(prompt.format_messages())
        return 'true' in response.content.lower()
    
    def run(self):
        """Main interaction loop"""
        print("Welcome to the Quiz Chatbot!")
        
        while True:
            print("\nAvailable categories:")
            print(self.get_categories())
            
            # Get category selection
            selection = input("\nPlease select a category number (or 'quit' to exit): ")
            
            if selection.lower() == 'quit':
                print("Thank you for playing!")
                break
                
            try:
                category_index = int(selection) - 1
                if 0 <= category_index < len(self.categories):
                    selected_category = self.categories[category_index]
                    
                    # Get random question from category
                    question, answer = self.select_question(selected_category)
                    if question is None:
                        print("Error: No questions available in this category.")
                        continue
                    
                    # Ask question and get user's answer
                    print(f"\nQuestion: {question}")
                    user_answer = input("Your answer: ")
                    
                    # Verify answer
                    is_correct = self.verify_answer(user_answer, answer)
                    
                    # Provide feedback
                    if is_correct:
                        print("Correct! Well done!")
                    else:
                        print(f"Incorrect. The correct answer was: {answer}")
                else:
                    print("Invalid category number. Please try again.")
            except ValueError:
                print("Please enter a valid number.")

# Usage
if __name__ == "__main__":
    chatbot = QuizChatbot("test_data/test_question_set_cleaned.csv")
    chatbot.run()


Welcome to the Quiz Chatbot!

Available categories:
1. HISTORY
2. ESPN's TOP 10 ALL-TIME ATHLETES
3. EVERYBODY TALKS ABOUT IT...
4. THE COMPANY LINE
5. EPITAPHS & TRIBUTES
6. 3-LETTER WORDS
7. MEASURING DEVICES
8. MYTHOLOGY
9. TELEVISION
10. ANNUAL EVENTS
11. HOMOPHONIC PAIRS
12. AMERICAN EXPLORERS
13. BALLET
14. U.S. MONEY
15. ISLANDS
16. CALENDARS
17. CONGRESS
18. PEOPLE IN HISTORY
19. FILM STARS
20. LAW
21. U.S. STATES
22. OF NO IMPORTANCE
23. GOLF TALK
24. THE GOSPELS
25. THAT '70s TEAM
26. "CO" CO.s
27. WOMEN'S FIRSTS
28. CORN-UCOPIA
29. ANIMATED PUZZLES
30. GENERAL SCIENCE
31. CARTOON VOICES
32. MY NAME IS HENRY, I'LL BE YOUR WRITER
33. VARIETY PACK
34. OHIOANS
35. "BUCK" EYES
36. FORGOTTEN MUSICALS
37. READ THE BOOK, SAW THE FILM
38. TOUGH HODGEPODGE
39. "LET"s END THIS
40. THE AMERICAN REVOLUTION
41. BY GEORGE!

Question: In the winter of 1971-72, a record 1,122 inches of snow fell at Rainier Paradise Ranger Station in this state
Incorrect. The correct answer was: Washington

A

In [12]:
import pandas as pd
from langchain.agents import Tool
from langchain.agents import AgentExecutor, CasualAgent
from langchain.memory import ConversationBufferMemory
import random

class QuizBot:
    def __init__(self, csv_path):
        # Load the CSV data
        self.df = pd.read_csv(csv_path)
        self.categories = self.df['category'].unique().tolist()
        self.current_question = None
        self.current_answer = None
        
        # Initialize LangChain components
        self.llm = ChatOllama(
            model="phi3:3.8b",
            temperature=0
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history")
        
        # Define tools for the agent
        self.tools = [
            Tool(
                name="get_categories",
                func=self.get_categories,
                description="Get list of available categories"
            ),
            Tool(
                name="select_question",
                func=self.select_question,
                description="Select a random question from a category"
            ),
            Tool(
                name="verify_answer",
                func=self.verify_answer,
                description="Verify if the provided answer is correct"
            )
        ]
        
        # Create the agent
        self.agent = ConversationationalAgent.from_llm_and_tools(
            llm=self.llm,
            tools=self.tools,
            verbose=True
        )
        
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True
        )

    def get_categories(self):
        """Return available categories"""
        return f"Available categories: {', '.join(self.categories)}"

    def select_question(self, category):
        """Select a random question from the specified category"""
        category_questions = self.df[self.df['Category'] == category]
        if len(category_questions) == 0:
            return "Category not found"
        
        question_row = category_questions.sample(n=1).iloc[0]
        self.current_question = question_row['Question']
        self.current_answer = question_row['Answer']
        return self.current_question

    def verify_answer(self, user_answer):
        """Verify if the user's answer is correct"""
        if self.current_answer is None:
            return "No question has been selected yet"
        
        # Use the LLM to compare answers more flexibly
        prompt = f"""
        Compare these answers and determine if they are semantically equivalent:
        Correct answer: {self.current_answer}
        User answer: {user_answer}
        Respond with only 'True' if they match, 'False' if they don't.
        """
        
        response = self.llm.predict(prompt)
        is_correct = 'true' in response.lower()
        
        return "Correct!" if is_correct else f"Incorrect. The correct answer is: {self.current_answer}"

    def run(self):
        """Run the interactive quiz"""
        print("Welcome to the Quiz Bot!")
        print(self.get_categories())
        
        while True:
            user_input = input("\nPlease select a category or type 'quit' to exit: ")
            
            if user_input.lower() == 'quit':
                break
                
            if user_input in self.categories:
                question = self.select_question(user_input)
                print(f"\nQuestion: {question}")
                
                answer = input("Your answer: ")
                result = self.verify_answer(answer)
                print(result)
            else:
                print("Invalid category. Please choose from the available categories.")

# Usage
if __name__ == "__main__":
    quiz_bot = QuizBot("test_data/test_question_set_cleaned.csv")
    quiz_bot.run()


ImportError: cannot import name 'CasualAgent' from 'langchain.agents' (/opt/anaconda3/envs/dev/lib/python3.10/site-packages/langchain/agents/__init__.py)